# Multi-Dimensional ICBS for VER Resolution

A single pool for resolving markets with multiple outcomes using VER tokens.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

## Multi-ICBS: One Pool, Many Outcomes

In [ ]:
class MultiICBS:
    """
    Multi-outcome ICBS pool for VER-based resolution.
    All outcome tokens trade in a single pool.
    """
    
    def __init__(self, outcomes, bootstrap_ver=100):
        self.outcomes = outcomes
        self.n = len(outcomes)
        
        # Start with equal supplies
        initial_supply = bootstrap_ver / self.n
        self.supplies = {outcome: initial_supply for outcome in outcomes}
        
        # Lambda scales with number of outcomes
        self.lambda_val = np.sqrt(self.n)
        
        # Track VER in pool
        self.ver_locked = bootstrap_ver
    
    def get_price(self, outcome):
        """Price of an outcome token in VER"""
        total_norm = np.sqrt(sum(s**2 for s in self.supplies.values()))
        if total_norm == 0:
            return 0
        return self.lambda_val * self.supplies[outcome] / total_norm
    
    def get_all_prices(self):
        """Get all outcome prices"""
        return {outcome: self.get_price(outcome) for outcome in self.outcomes}
    
    def trade(self, buy_outcome, ver_amount):
        """
        Trade VER for outcome tokens.
        Simple version - just increases supply of bought outcome.
        """
        # Calculate how many tokens for this VER
        current_price = self.get_price(buy_outcome)
        if current_price > 0:
            tokens_received = ver_amount / current_price * 0.9  # 10% slippage
            self.supplies[buy_outcome] += tokens_received
            self.ver_locked += ver_amount
            
            # Rebalance to maintain ICBS constraint
            self._rebalance()
    
    def _rebalance(self):
        """Adjust supplies to maintain cost function"""
        # Keep total cost = VER locked
        current_cost = self.lambda_val * np.sqrt(sum(s**2 for s in self.supplies.values()))
        if current_cost > 0:
            scale = self.ver_locked / current_cost
            for outcome in self.outcomes:
                self.supplies[outcome] *= scale
    
    def get_winner(self):
        """Returns outcome with highest price"""
        prices = self.get_all_prices()
        return max(prices, key=prices.get)

In [ ]:
# Example: NBA Finals Resolution
pool = MultiICBS(
    outcomes=["LAKERS", "CELTICS", "HEAT", "NUGGETS"],
    bootstrap_ver=100
)

print("Initial state:")
for outcome, price in pool.get_all_prices().items():
    print(f"  {outcome}: {price:.3f} VER")
print(f"  Sum of prices: {sum(pool.get_all_prices().values()):.3f} (should be ~{pool.lambda_val:.3f})")

# Simulate VER holders believing Lakers won
print("\nAfter VER holders trade (believing Lakers won):")
pool.trade("LAKERS", 50)  # Big buy
pool.trade("LAKERS", 30)  # Another buy
pool.trade("CELTICS", 5)  # Small contrarian bet

for outcome, price in pool.get_all_prices().items():
    print(f"  {outcome}: {price:.3f} VER")
    
print(f"\nWinner: {pool.get_winner()}")

In [ ]:
def visualize_resolution(trades_data):
    """
    Visualize how prices evolve as VER holders trade
    """
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Price Evolution", "Final Prices"),
        specs=[[{"type": "scatter"}, {"type": "bar"}]]
    )
    
    # Simulate trades
    pool = MultiICBS(["A", "B", "C", "D"], bootstrap_ver=100)
    history = []
    
    for i, (outcome, amount) in enumerate(trades_data):
        pool.trade(outcome, amount)
        prices = pool.get_all_prices()
        for out, price in prices.items():
            history.append({"step": i+1, "outcome": out, "price": price})
    
    df = pd.DataFrame(history)
    
    # Plot price evolution
    for outcome in pool.outcomes:
        data = df[df.outcome == outcome]
        fig.add_trace(
            go.Scatter(x=data.step, y=data.price, name=outcome, mode='lines+markers'),
            row=1, col=1
        )
    
    # Plot final prices
    final_prices = pool.get_all_prices()
    fig.add_trace(
        go.Bar(x=list(final_prices.keys()), y=list(final_prices.values())),
        row=1, col=2
    )
    
    fig.update_layout(height=400, showlegend=True)
    fig.update_xaxes(title_text="Trade #", row=1, col=1)
    fig.update_xaxes(title_text="Outcome", row=1, col=2)
    fig.update_yaxes(title_text="Price (VER)", row=1, col=1)
    fig.update_yaxes(title_text="Price (VER)", row=1, col=2)
    
    fig.show()
    
    print(f"\nResolution: {pool.get_winner()} wins!")
    return pool

# Example trades (outcome, VER amount)
trades = [
    ("A", 30),  # Someone believes A
    ("A", 20),  # More belief in A
    ("B", 10),  # Small bet on B
    ("A", 40),  # Strong conviction in A
    ("C", 5),   # Tiny contrarian bet
]

pool = visualize_resolution(trades)